In [15]:
%pip install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 54.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=01ac3c0a386f2827840997daab1176bc233638b5d735f40d5811d025e2620aaf
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [6]:
from datasets import load_dataset
from itertools import islice
from google.colab import userdata
api = userdata.get('HF_TOKEN')
ds = load_dataset(
    "Lichess/standard-chess-games",
    split="train",
    streaming=True,
    token=api
)



Resolving data files:   0%|          | 0/26138 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26138 [00:00<?, ?it/s]

In [7]:
ds

IterableDataset({
    features: ['Event', 'Site', 'White', 'Black', 'Result', 'WhiteTitle', 'BlackTitle', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'UTCDate', 'UTCTime', 'ECO', 'Opening', 'Termination', 'TimeControl', 'movetext'],
    num_shards: 26138
})

In [8]:
ds = ds.select_columns(['movetext',
    "WhiteElo",
    "BlackElo",
    "WhiteRatingDiff",
    "BlackRatingDiff",
    "Opening",
    "ECO"])

In [9]:
ds

IterableDataset({
    features: ['movetext', 'WhiteElo', 'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'Opening', 'ECO'],
    num_shards: 26138
})

In [38]:
len(small_ds)

2000

In [12]:
ds = ds.rename_columns({
    "WhiteElo": "white_elo",
    "BlackElo": "black_elo",
    "WhiteRatingDiff": "white_diff",
    "BlackRatingDiff": "black_diff",
    "Opening": "opening",
    "ECO": "eco"
})


In [14]:
ds.features

{'movetext': Value('string'),
 'white_elo': Value('int16'),
 'black_elo': Value('int16'),
 'white_diff': Value('int16'),
 'black_diff': Value('int16'),
 'opening': Value('string'),
 'eco': Value('string')}

In [16]:
import re

def clean_movetext(movetext: str):
    # remove result like 1-0, 0-1, 1/2-1/2
    movetext = re.sub(r"1-0|0-1|1/2-1/2", "", movetext)

    # remove move numbers like "1.", "23..."
    movetext = re.sub(r"\d+\.", "", movetext)

    # split into tokens
    moves = movetext.strip().split()

    return moves


In [26]:
import chess

def generate_samples(movetext):
    """
    Returns list of (board_fen, move_uci)
    """
    board = chess.Board()
    samples = []

    moves = clean_movetext(movetext)
    print(moves)
    for san in moves:
        try:
            move = board.parse_san(san)
        except Exception:
            # invalid or corrupted move → skip game
            return []

        # save state BEFORE move
        board_fen = board.fen()
        move_uci = move.uci()

        samples.append((board_fen, move_uci))

        board.push(move)

    return samples


In [27]:
sample = next(iter(ds))

samples = generate_samples(sample["movetext"])

print("Number of move-samples:", len(samples))
print("First sample:")
print("Board FEN:", samples[0][0])
print("Next move (UCI):", samples[0][1])


['e4', 'e6', 'd4', 'b6', 'a3', 'Bb7', 'Nc3', 'Nh6', 'Bxh6', 'gxh6', 'Be2', 'Qg5', 'Bg4', 'h5', 'Nf3', 'Qg6', 'Nh4', 'Qg5', 'Bxh5', 'Qxh4', 'Qf3', 'Kd8', 'Qxf7', 'Nc6', 'Qe8#']
Number of move-samples: 25
First sample:
Board FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Next move (UCI): e2e4


In [34]:
all_samples = []

for i, row in enumerate(ds):
    game_samples = generate_samples(row["movetext"])
    all_samples.extend(game_samples)

    if i == 5:   # only first 5 games for testing
        break

print("Total samples collected:", len(all_samples))


['e4', 'e6', 'd4', 'b6', 'a3', 'Bb7', 'Nc3', 'Nh6', 'Bxh6', 'gxh6', 'Be2', 'Qg5', 'Bg4', 'h5', 'Nf3', 'Qg6', 'Nh4', 'Qg5', 'Bxh5', 'Qxh4', 'Qf3', 'Kd8', 'Qxf7', 'Nc6', 'Qe8#']
['d4', 'd5', 'Nf3', 'Nf6', 'e3', 'Bf5', 'Nh4', 'Bg6', 'Nxg6', 'hxg6', 'Nd2', 'e6', 'Bd3', 'Bd6', 'e4', 'dxe4', 'Nxe4', 'Rxh2', 'Ke2', 'Rxh1', 'Qxh1', 'Nc6', 'Bg5', 'Ke7', 'Qh7', 'Nxd4+', 'Kd2', 'Qe8', 'Qxg7', 'Qh8', 'Bxf6+', 'Kd7', 'Qxh8', 'Rxh8', 'Bxh8']
['e4', 'e5', 'Nf3', 'Nc6', 'Bc4', 'Nf6', 'Nc3', 'Bc5', 'a3', 'Bxf2+', 'Kxf2', 'Nd4', 'd3', 'Ng4+', 'Kf1', 'Qf6', 'h3', 'd5', 'Nxd5', 'Qe6', 'Nxc7+']
['e4', 'c6', 'Nc3', 'd5', 'Qf3', 'dxe4', 'Nxe4', 'Nd7', 'Bc4', 'Ngf6', 'Nxf6+', 'Nxf6', 'Qg3', 'Bf5', 'd3', 'Bg6', 'Ne2', 'e6', 'Bf4', 'Nh5', 'Qf3', 'Nxf4', 'Nxf4', 'Be7', 'Bxe6', 'fxe6', 'Nxe6', 'Qa5+', 'c3', 'Qe5+', 'Qe3', 'Qxe3+', 'fxe3', 'Kd7', 'Nf4', 'Bd6', 'Nxg6', 'hxg6', 'h3', 'Bg3+', 'Kd2', 'Raf8', 'Rhf1', 'Ke7', 'd4', 'Rxf1', 'Rxf1', 'Rf8', 'Rxf8', 'Kxf8', 'e4', 'Ke7', 'Ke3', 'g5', 'Kf3', 'Be1', 'Kg4', 'Bd2

In [35]:
all_samples

[('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1', 'e2e4'),
 ('rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1', 'e7e6'),
 ('rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2', 'd2d4'),
 ('rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBNR b KQkq - 0 2', 'b7b6'),
 ('rnbqkbnr/p1pp1ppp/1p2p3/8/3PP3/8/PPP2PPP/RNBQKBNR w KQkq - 0 3', 'a2a3'),
 ('rnbqkbnr/p1pp1ppp/1p2p3/8/3PP3/P7/1PP2PPP/RNBQKBNR b KQkq - 0 3', 'c8b7'),
 ('rn1qkbnr/pbpp1ppp/1p2p3/8/3PP3/P7/1PP2PPP/RNBQKBNR w KQkq - 1 4', 'b1c3'),
 ('rn1qkbnr/pbpp1ppp/1p2p3/8/3PP3/P1N5/1PP2PPP/R1BQKBNR b KQkq - 2 4', 'g8h6'),
 ('rn1qkb1r/pbpp1ppp/1p2p2n/8/3PP3/P1N5/1PP2PPP/R1BQKBNR w KQkq - 3 5',
  'c1h6'),
 ('rn1qkb1r/pbpp1ppp/1p2p2B/8/3PP3/P1N5/1PP2PPP/R2QKBNR b KQkq - 0 5', 'g7h6'),
 ('rn1qkb1r/pbpp1p1p/1p2p2p/8/3PP3/P1N5/1PP2PPP/R2QKBNR w KQkq - 0 6', 'f1e2'),
 ('rn1qkb1r/pbpp1p1p/1p2p2p/8/3PP3/P1N5/1PP1BPPP/R2QK1NR b KQkq - 1 6',
  'd8g5'),
 ('rn2kb1r/pbpp1p1p/1p2p2p/6q1/3PP3/P1N5/1PP1BPPP/R2QK1NR w KQkq - 

In [20]:
import chess
import numpy as np

PIECE_TO_CHANNEL = {
    chess.PAWN:   0,
    chess.KNIGHT: 1,
    chess.BISHOP: 2,
    chess.ROOK:   3,
    chess.QUEEN:  4,
    chess.KING:   5
}

def board_to_tensor(fen: str):
    board = chess.Board(fen)
    tensor = np.zeros((12, 8, 8), dtype=np.float32)

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is None:
            continue

        row = 7 - (square // 8)
        col = square % 8

        channel = PIECE_TO_CHANNEL[piece.piece_type]
        if piece.color == chess.BLACK:
            channel += 6

        tensor[channel, row, col] = 1.0

    return tensor


In [21]:
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"

tensor = board_to_tensor(fen)

print(tensor.shape)          # (12, 8, 8)
print(tensor.sum())          # 32 pieces


(12, 8, 8)
32.0


In [50]:
import pandas as pd
small_ds = list(islice(ds, 4000))
df = pd.DataFrame(small_ds)
len(df)

4000

In [ ]:
df.to_csv("small_dataset.csv", index=False)